# Local Enhanced Global Ensemble Modeling (LEGEN-DTM): local modeling notebook

The notebook describes a global-to-local model example in the research of GEDTM30 (https://github.com/openlandmap/GEDTM30). A 1 degree x 1 degree tile is predicted through

1. Global model:

GEDTM30 global model is trained by globally stratified ICESat-2 and GEDI terrain height samples.

2. Local model:

Additional samples (samples.csv) from a 5 degree x 5 degree ICESat-2 and GEDI samples that covers this 1 degree tile are used to enhance the global model to a local model with 100 additional trees upon the 100 trees from the global model.

We will go through a detail in reproducing the locally enhanced modeling process. That includes:

(1) Rewrite "RandomForestRegressor" from sklearn to return individual trees output so as to derive terrain prediction and uncertainty (stanrd deviation)

(2) Load covariates from online Zenodo bucket via HTTP range request

(3) Predict a global model 

(4) Build additional trees to the global model in order to obtain a local-enhanced model (global-to-local modeling)

(5) Predict a local model

This notebook is the reproducible script powered by Zenodo, from the repository (https://zenodo.org/records/14914836) which stores the covariates in COG format and the global model (file_name). The script is designed to run in local independently. 



In [ ]:
# import libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation import check_is_fitted
import rasterio
import numpy as np
import joblib
import pandas as pd
import bottleneck as bn
import threading
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from tqdm import tqdm

## Part 1: Rewrite "RandomForestRegressor" from sklearn to return individual trees output so as to derive terrain prediction and uncertainty (stanrd deviation)

In [ ]:
# define functions 
def _single_prediction(predict, X, out, i, lock):
    prediction = predict(X, check_input=False)
    with lock:
        out[i, :] = prediction

def cast_tree_rf(model):
    model.__class__ = TreesRandomForestRegressor
    return model

class TreesRandomForestRegressor(RandomForestRegressor):
    def predict(self, X):
        """
        Predict regression target for X.

        The predicted regression target of an input sample is computed according
        to a list of functions that receives the predicted regression targets of each 
        single tree in the forest.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            The input samples. Internally, its dtype will be converted to
            ``dtype=np.float32``. If a sparse matrix is provided, it will be
            converted into a sparse ``csr_matrix``.

        Returns
        -------
        s : an ndarray of shape (n_estimators, n_samples)
            The predicted values for each single tree.
        """
        check_is_fitted(self)
        # Check data
        X = self._validate_X_predict(X)

        # store the output of every estimator
        assert(self.n_outputs_ == 1)
        pred_t = np.empty((len(self.estimators_), X.shape[0]), dtype=np.float32)
        # Assign chunk of trees to jobs
        n_jobs = min(self.n_estimators, self.n_jobs)
        # Parallel loop prediction
        lock = threading.Lock()
        Parallel(n_jobs=n_jobs, verbose=self.verbose, require="sharedmem")(
            delayed(_single_prediction)(self.estimators_[i].predict, X, pred_t, i, lock)
            for i in range(len(self.estimators_))
        )
        return pred_t

## Part 2: Load covariates from online Zenodo bucket via HTTP range request

In [ ]:
files=['https://zenodo.org/records/14914836/files/slope_etopo2022.tiff?download=1',
'https://zenodo.org/records/14914836/files/canopy.height_glad.tiff?download=1',
'https://zenodo.org/records/14914836/files/canopy.height_eth.tiff?download=1',
'https://zenodo.org/records/14914836/files/dsm_aw3d30.tiff?download=1',
'https://zenodo.org/records/14914836/files/dsm_glo30.tiff?download=1',
'https://zenodo.org/records/14914836/files/edge.canopy.height_glad.tiff?download=1',
'https://zenodo.org/records/14914836/files/edge.canopy.height_eth.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndvi.p025_2006.2010tiff?download=1',
'https://zenodo.org/records/14914836/files/ndvi.p025_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndvi.p50_2006.2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndvi.p50_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndvi.p975_2006.2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndvi.p975_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndwi.p025_2006.2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndwi.p025_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndwi.p50_2006.2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndwi.p50_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndwi.p975_2006.2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/ndwi.p975_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/nir.p025_2006.2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/nir.p025_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/nir.p50_2006.2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/nir.p50_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/nir.p975_2006_2010.tiff?download=1',
'https://zenodo.org/records/14914836/files/nir.p975_2011.2015.tiff?download=1',
'https://zenodo.org/records/14914836/files/tree.cover_glad.tiff?download=1',
'https://zenodo.org/records/14914836/files/building.height_3dglobfp.tiff?download=1',
'https://zenodo.org/records/14914836/files/building.height_ghsbuilth.tiff?download=1',
'https://zenodo.org/records/14914836/files/building.extent_wsf2019.tiff?download=1',
'https://zenodo.org/records/14914836/files/lcluc.change_glad.tiff?download=1']

In [ ]:
data_list=[]
for file in tqdm(files):
    data=rasterio.open(file).read(1)
    data_list.append(data)

In [ ]:
input_data=np.dstack(data_list)

In [ ]:
x_size,y_size,layers=input_data.shape

In [ ]:
input_data.transpose(2,0,1).shape

In [ ]:
data = input_data.transpose(2,0,1).reshape(layers,-1)

In [ ]:
# extract the permanent ice from lc_glad.glcluc.change
lulc=data[-1,:]
permanent_ice=lulc==241
data=data[:-1,:]

# assign aw3d30 value on glo30 in azerbaijan_mask
azerbaijan_mask=np.isnan(data[4,:])
data[4,azerbaijan_mask]=data[3,azerbaijan_mask]
build_mask=np.isnan(data[-2,:])
data[-2,build_mask]=data[-3,build_mask]

# give canopy to 0 if it is on permanent ice pixel
data[1,permanent_ice]=0
data[2,permanent_ice]=0

# clean up huge discrepancy from DSMs
d=10 # threshold for terrain difference
alos_sub_mask=abs(data[3,:]-data[4,:])>d
data[3,alos_sub_mask]=data[4,alos_sub_mask]

# set nan data to 0 (no builing, no canopy, etc...)
data[np.isnan(data)] = 0
data=data.transpose(1,0)

## Part 3: Predict a global model

In [ ]:
# download the global model
!wget https://zenodo.org/records/14914777/files/global.model_gedtm30.lz4?download=1 -O global.model_gedtm30.lz4

In [ ]:
# load the model
m = joblib.load('global.model_gedtm30.lz4')['model_rf']
# change the class to return individual trees
m.__class__ = TreesRandomForestRegressor
m

In [ ]:
# predict the terrain height and obtain standard devation through the global model
y_rf = m.predict(data)  
print(y_rf.shape) # (tree, pixel)
gpredictions = bn.nanmean(y_rf, axis=0)*10 # scaling to decimeter
gstd = bn.nanstd(y_rf, axis=0)*100 # scaling to millimeter

In [ ]:
# visualize the result
plt.imshow(gpredictions.reshape(1,x_size,y_size)[0,:,:])

In [ ]:
# save the result

template = rasterio.open(files[5])
kwargs = template.meta
kwargs['compress'] = 'deflate'

kwargs['dtype']= rasterio.int32
kwargs['nodata']=2147483647
with rasterio.open('gedtm_pred.tif', 'w', **kwargs) as dst:
    dst.write(gpredictions.reshape(1,x_size,y_size)[0,:,:],1)
    
kwargs['dtype']= rasterio.int16
kwargs['nodata']=32767
with rasterio.open('gedtm_pred_std.tif', 'w', **kwargs) as dst:
    dst.write(gstd.reshape(1,x_size,y_size)[0,:,:],1)

## Part 4: Build additional trees to the global model in order to obtain a local-enhanced model (global-to-local modeling)


In [ ]:
# download the additional local samples
!wget https://zenodo.org/records/14914777/files/local_samples.csv?download=1 -O local_samples.csv

In [ ]:
# load the local samples
samples = pd.read_csv('local_samples.csv',index_col=0)

In [ ]:
# compare with global model feature importance
feature_importances = pd.Series(m.feature_importances_, index=samples.columns[:-1])
print(feature_importances.sort_values(ascending=False))

In [ ]:
# add 100 estimators (total 200) for local fine tuning
m.set_params(n_estimators=200, warm_start=True)
m.fit(samples[samples.columns[:-1]], samples['dtm_y'])

In [ ]:
# local enhanced global model feature importance
feature_importances = pd.Series(m.feature_importances_, index=samples.columns[:-1])
print(feature_importances.sort_values(ascending=False))

In [ ]:
# predict the terrain height and obtain standard devation through the locally enhanced model
y_rf = m.predict(data) 
print(y_rf.shape) # (tree, pixel)
lpredictions = bn.nanmean(y_rf, axis=0)*10 # scaling to decimeter
lstd = bn.nanstd(y_rf, axis=0)*100 # scaling to millimeter

In [ ]:
# visualize the result
plt.imshow(lpredictions.reshape(1,x_size,y_size)[0,:,:])

In [ ]:
# save the result
kwargs['dtype']= rasterio.int32
kwargs['nodata']=2147483647
with rasterio.open('legdtm_pred.tif', 'w', **kwargs) as dst:
    dst.write(lpredictions.reshape(1,x_size,y_size)[0,:,:],1)

kwargs['dtype']= rasterio.int32
kwargs['nodata']=32767
with rasterio.open('legdtm_pred_std.tif', 'w', **kwargs) as dst:
    dst.write(lstd.reshape(1,x_size,y_size)[0,:,:],1)